In [30]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import fbeta_score, precision_recall_curve, confusion_matrix


#from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [3]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [4]:
# train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [5]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]


In [6]:
def threshold_by_fbeta(y_test: pd.Series, y_pred: list, *, beta: int = 1) -> tuple:
    
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
    fbeta = ((1 + beta**2) * precision * recall) / (beta**2 * precision + recall)
    index = np.argmax(fbeta)
    
    return thresholds[index], fbeta[index]

In [7]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continious_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']


In [8]:
final_transformers = []

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continious_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', MinMaxScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [9]:
feats = FeatureUnion(final_transformers)

In [10]:
# creating models

In [11]:
classifiers = [
    RandomForestClassifier(random_state=42),
    GradientBoostingClassifier(random_state=42),
    LogisticRegression(random_state=42)
]

In [12]:
pipelines = []
metrics = {'threshold': [], 'fbeta': []}

In [13]:
for clf in classifiers:
    pipeline = Pipeline([
        ('features', feats),
        ('classifier', clf),
    ])
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict_proba(X_test)[:, 1]
    threshold, fbeta_score = threshold_by_fbeta(y_test, y_pred, beta=2)
    
    metrics['threshold'].append(threshold)
    metrics['fbeta'].append(fbeta_score)
    pipelines.append(pipeline)

In [14]:
metrics_df = pd.DataFrame(metrics, index=[clf.__class__.__name__ for clf in classifiers])
metrics_df.sort_values('fbeta', ascending=False)
# metrics_df

,threshold,fbeta
GradientBoostingClassifier,0.154589,0.717822
RandomForestClassifier,0.150000,0.693304
LogisticRegression,0.135475,0.636624


In [19]:
best_model = pipelines[1]
best_model

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Tenu...
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Balance')),
                                                                 ('scaler',
                                                                  MinMaxScaler())])),
                                                ('NumOfProducts',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='NumOfProducts')),
                                                                 ('scaler',
                                                                  MinMaxScaler())])),
                                                ('EstimatedSalary',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='EstimatedSalary')),
                                                                 ('scaler',
                                                                  MinMaxScaler())]))])),
                ('classifier', GradientBoostingClassifier(random_state=42))])

Lets estimate economic efficacy of the best model:

In [25]:
y_pred_best = best_model.predict_proba(X_test)[:, 1]
_, _, thresholds = precision_recall_curve(y_test, y_pred)

In [26]:
def economic_estimator(y_pred_probas: list, thresholds: list) -> pd.DataFrame:
    profits = []
    expenses = []
    for thr in thresholds:
        preds = (y_pred_best >= thr).astype(bool)
        conf = confusion_matrix(y_test, preds)
        TP = conf[1][1]
        FP = conf[0][1]
        profits.append(TP * 2)
        expenses.append(FP * -1)

    results_ = pd.DataFrame({'threshold': thresholds, 'profit': profits, 'expense': expenses})
    results_['total'] = results_['profit'] + results_['expense']
    return results_

In [27]:
results_ = economic_estimator(y_pred_best, thresholds)
results_.sort_values('total', ascending=False)

,threshold,profit,expense,total
2178,0.407250,608,-128,480
2175,0.406141,608,-128,480
2179,0.407268,608,-128,480
2180,0.407311,608,-128,480
2177,0.406888,608,-128,480
...,...,...,...,...
4,0.014457,1014,-1960,-946
3,0.014286,1014,-1963,-949
2,0.013231,1016,-1983,-967
1,0.011778,1018,-1989,-971


In [52]:
results_['total'].sum()

411994

In [34]:
params = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__min_samples_leaf': [1, 2, 3],
    'classifier__max_depth': [3, 4, 5]
}

In [35]:
grid = GridSearchCV(best_model,
                    param_grid=params,
                    cv=5,
                    refit=False)

In [36]:
search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__max_depth': 5,
 'classifier__min_samples_leaf': 3,
 'classifier__n_estimators': 100}

In [39]:
optimal_model = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(n_estimators=100, max_depth=5, min_samples_leaf=3, random_state=42)),
])


In [40]:
optimal_model.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Tenu...
                                                                  NumberSelector(key='Balance')),
                                                                 ('scaler',
                                                                  MinMaxScaler())])),
                                                ('NumOfProducts',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='NumOfProducts')),
                                                                 ('scaler',
                                                                  MinMaxScaler())])),
                                                ('EstimatedSalary',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='EstimatedSalary')),
                                                                 ('scaler',
                                                                  MinMaxScaler())]))])),
                ('classifier',
                 GradientBoostingClassifier(max_depth=5, min_samples_leaf=3,
                                            random_state=42))])

In [42]:
y_pred_optim = optimal_model.predict_proba(X_test)[:, 1]    


In [43]:
threshold, fbeta_score = threshold_by_fbeta(y_test, y_pred_optim, beta=2)


In [44]:
threshold, fbeta_score


(0.12823119166694474, 0.7170418006430869)

Threshold is lower and fbeta_score is a bit lower than previous GB model with set parameters

Lets estimate economic efficacy of the model:

In [50]:
, _, thresholds_optim = precision_recall_curve(y_test, y_pred_optim)

results_optim = economic_estimator(y_pred_best, thresholds_optim)
results_optim.sort_values('total', ascending=False)

,threshold,profit,expense,total
2036,0.404966,608,-128,480
2035,0.404927,608,-128,480
2037,0.406520,608,-128,480
2038,0.407940,608,-128,480
2039,0.409099,606,-127,479
...,...,...,...,...
20,0.009944,1018,-1991,-973
21,0.010006,1018,-1991,-973
22,0.010161,1018,-1991,-973
23,0.010165,1018,-1991,-973


In [54]:
results_optim.sum()

threshold    5.428720e+02
profit       2.031404e+06
expense     -2.259688e+06
total       -2.282840e+05
dtype: float64

In [53]:
results_optim['total'].sum()

-228284

This model is not profitable. We should keep threshold higher like in original model GB.